# X3FL dynamic relationships

## The relationship

In [1]:
import numpy as np
import pandas as pd

In [2]:
all_races = ["Teladi", "NMMC", "Goner", "TerraCorp", "Strong Arms", "Argon", "Boron", "Split", "Arteus", "OTAS", "Duke's", "Paranid", "Pirates", "Terran", "Yaki"]
# https://imgur.com/nqI0nbO, https://imgur.com/7BhbUMa, by blazenclaw, forum.egosoft.com
try:
    relationship_df = pd.read_csv("relationship.csv", index_col=0)
except:
    relationship_df = pd.read_csv("https://raw.githubusercontent.com/mkmark/X3FL-dynamic-relationships/main/relationship.csv", index_col=0)
relationship_df = pd.DataFrame(relationship_df, columns=all_races)
relationship_df

,Teladi,NMMC,Goner,TerraCorp,Strong Arms,Argon,Boron,Split,Arteus,OTAS,Duke's,Paranid,Pirates,Terran,Yaki
Teladi,1.00,0.15,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,-0.30
NMMC,0.15,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,-0.30,0.00,0.00,0.0,0.00
Goner,0.00,0.00,1.00,0.15,-0.30,0.10,0.05,0.00,0.00,0.00,-0.30,0.00,-0.30,0.0,-0.15
TerraCorp,0.00,0.00,0.15,1.00,-0.15,0.15,0.05,-0.15,0.05,-0.30,0.00,-0.30,-0.30,0.0,-0.30
Strong Arms,0.00,0.00,0.00,-0.15,1.00,-0.15,-0.30,0.15,-0.30,-0.15,0.00,0.05,0.00,-0.3,0.00
Argon,0.00,0.00,0.10,0.15,-0.15,1.00,0.15,-0.15,0.05,0.15,-0.30,-0.30,-0.30,-0.3,-0.30
Boron,0.00,0.00,0.05,0.05,-0.30,0.15,1.00,-0.30,0.15,0.05,-0.30,-0.15,-0.30,-0.3,-0.30
Split,0.00,0.00,0.00,-0.15,0.15,-0.15,-0.30,1.00,-0.30,-0.15,-0.30,0.15,0.00,-0.3,-0.30
Arteus,0.00,0.00,0.00,0.05,-0.30,0.05,0.15,-0.30,1.00,-0.30,0.00,-0.15,-0.30,-0.3,-0.30
OTAS,0.00,0.00,0.00,-0.30,-0.15,0.15,0.05,-0.15,-0.30,1.00,0.00,-0.30,-0.30,-0.3,-0.30


The problem is simple: given the relationship $\mathbf{R}$, target notority $\mathbf{N}$, find tactics $\mathbf{X}$, which is a vector that represents how much notority point one should get from a race.

$$
\mathbf{R}\mathbf{X}=\mathbf{N}
$$

$$
\mathbf{X}=\mathbf{R}^{-1}\mathbf{N}
$$

Let's first define $\mathbf{N}$ as vectors of 1, since we don't want to lose effort by hitting the notoriety ceiling.

In [3]:
def get_X(races):
    R = np.array(relationship_df.loc[races, races])
    invR = np.linalg.inv(R) 
    N = np.ones(len(races))
    X = invR@N
    return X

def get_X_df(races):
    X = get_X(races)
    return pd.DataFrame(list(X)+[sum(X)]+[len(races)/sum(X)], index=races+['sum', 'efficiency']).transpose()

These races selected are the control group of races that are friends, but not necessarily to be the only friends as an notoriety overflow is allowed, which will make $N_i > 1$

To calculate the possible solutions, one has to recuse through all subsets and check if some get possible solutions, see below

## Example solutions

Let's see if there is a solution for all the races

The number for each race represents how much effort you will have to put with that race to get 1 notoriety point for each race assuming you're to be friend with all these selected race.

The sum is the total workload of current tactic.

The efficiency is the sum of actual gained notoriety points (equal the number of races selected) divided by total workload.

In [4]:
get_X_df(all_races)

,Teladi,NMMC,Goner,TerraCorp,Strong Arms,Argon,Boron,Split,Arteus,OTAS,Duke's,Paranid,Pirates,Terran,Yaki,sum,efficiency
0,0.413665,0.834924,0.400697,-0.602183,-0.088362,-0.559179,-0.534538,-0.60492,-0.931595,-1.156118,-0.343422,-1.250485,-1.373975,-1.513874,-1.536987,-8.846352,-1.695614


Unfortunately there is not a solution as one would have to get negative points from races. The problem is, when actively losing points of a race, the notoriety points of enemy races does not increase accordingly. However if that is true, we can see a path that you do some missions for some races but keep robbing other races and one day you will find yourself master of diplomacy with all races - too good to be true for those races you robbed.

Let's get results of some interesting races combo.

In [5]:
commonwealth_races = ["Teladi", "Argon", "Boron", "Split", "Paranid"]
major_races = ["Teladi", "Argon", "Boron", "Split", "Paranid", "Terran"]
all_but_yaki_races = ["Teladi", "NMMC", "Goner", "TerraCorp", "Strong Arms", "Argon", "Boron", "Split", "Arteus", "OTAS", "Duke's", "Paranid", "Pirates", "Terran"]
all_but_yaki_terran_races = ["Teladi", "NMMC", "Goner", "TerraCorp", "Strong Arms", "Argon", "Boron", "Split", "Arteus", "OTAS", "Duke's", "Paranid", "Pirates"]
all_but_yaki_terran_pirates_races = ["Teladi", "NMMC", "Goner", "TerraCorp", "Strong Arms", "Argon", "Boron", "Split", "Arteus", "OTAS", "Duke's", "Paranid"]
all_but_yaki_pirates_races = ["Teladi", "NMMC", "Goner", "TerraCorp", "Strong Arms", "Argon", "Boron", "Split", "Arteus", "OTAS", "Duke's", "Paranid", "Terran"]
all_but_terran_pirates_races = ["Teladi", "NMMC", "Goner", "TerraCorp", "Strong Arms", "Argon", "Boron", "Split", "Arteus", "OTAS", "Duke's", "Paranid", "Yaki"]
all_but_teladi_yaki_terran_pirates_races = ["NMMC", "Goner", "TerraCorp", "Strong Arms", "Argon", "Boron", "Split", "Arteus", "OTAS", "Duke's", "Paranid"]

All commonwealth races, no problem

In [6]:
get_X_df(commonwealth_races)

,Teladi,Argon,Boron,Split,Paranid,sum,efficiency
0,1.0,1.428571,1.428571,1.428571,1.428571,6.714286,0.744681


All major races, no problem

In [7]:
get_X_df(major_races)

,Teladi,Argon,Boron,Split,Paranid,Terran,sum,efficiency
0,1.0,3.823529,3.823529,3.823529,3.823529,5.588235,21.882353,0.274194


All but Yaki, not an option

In [8]:
get_X_df(all_but_yaki_races)

,Teladi,NMMC,Goner,TerraCorp,Strong Arms,Argon,Boron,Split,Arteus,OTAS,Duke's,Paranid,Pirates,Terran,sum,efficiency
0,0.904032,0.639788,0.11671,-0.844907,-0.910454,-0.926158,-1.022721,-0.939952,-1.63911,-1.890735,-0.748691,-1.995335,-2.18678,-2.677981,-14.122294,-0.99134


According to calculation just 3 enemies seems not sufficient

In [9]:
get_X_df(all_but_yaki_terran_pirates_races)

,Teladi,NMMC,Goner,TerraCorp,Strong Arms,Argon,Boron,Split,Arteus,OTAS,Duke's,Paranid,sum,efficiency
0,-0.172497,7.816643,7.790471,13.40212,12.541359,10.368289,14.311163,17.984757,15.620415,18.506517,22.635896,21.638802,162.443937,0.073872


which is actually not the case since a subset is sufficient, just teledi notoriety points got overflowed doesn't render it illegal

In [10]:
get_X_df(all_but_teladi_yaki_terran_pirates_races)

,NMMC,Goner,TerraCorp,Strong Arms,Argon,Boron,Split,Arteus,OTAS,Duke's,Paranid,sum,efficiency
0,7.780909,7.781132,13.390187,12.529926,10.355119,14.294295,17.964491,15.606104,18.489653,22.60303,21.615287,162.410133,0.06773


In [11]:
R = np.array(relationship_df.loc[all_races, all_but_teladi_yaki_terran_pirates_races])
R@get_X(all_but_teladi_yaki_terran_pirates_races)

array([  1.16713635,   1.        ,   1.        ,   1.        ,
         1.        ,   1.        ,   1.        ,   1.        ,
         1.        ,   1.        ,   1.        ,   1.        ,
       -24.73473903, -40.03737145, -40.29544981])

thus our complete solution

In [12]:
# https://stackoverflow.com/questions/26332412/python-recursive-function-to-display-all-subsets-of-given-set
def subs(l):
    if l == []:
        return [[]]

    x = subs(l[1:])

    return x + [[l[0]] + y for y in x]

def get_X_optimal_sub(races):
    possible_solution_racess = []
    possible_solution_Xs = []
    possible_solution_workloads = []
    for sub_races in subs(races):
        X = get_X(sub_races)
        R_all = np.array(relationship_df.loc[races, sub_races])
        N_all = R_all@X
        if min(N_all)>0:
            possible_solution_racess += [sub_races]
            possible_solution_Xs += [X]
            possible_solution_workloads += [sum(X)]
    if len(possible_solution_workloads)>0:
        min_index = possible_solution_workloads.index(min(possible_solution_workloads))
        optiaml_solution_races = possible_solution_racess[min_index]
        optiaml_solution_X = possible_solution_Xs[min_index]
        return pd.DataFrame(list(optiaml_solution_X)+[sum(optiaml_solution_X)]+[len(optiaml_solution_races)/sum(optiaml_solution_X)], index=optiaml_solution_races+['sum', 'efficiency']).transpose()

In [13]:
get_X_optimal_sub(all_but_yaki_terran_pirates_races)

,NMMC,Goner,TerraCorp,Strong Arms,Argon,Boron,Split,Arteus,OTAS,Duke's,Paranid,sum,efficiency
0,7.780909,7.781132,13.390187,12.529926,10.355119,14.294295,17.964491,15.606104,18.489653,22.60303,21.615287,162.410133,0.06773


## Let's explore all possibilities

In [19]:
# all below is wrong and is to be updated

Turned out there are the following number of possible combos.

In [14]:
least_enemy_set_list = []
for races in subs(all_races):
    if races != []:
        if min(get_X(races))>0:
            enemy_set = set(all_races) - set(races)
            is_duplicate = False
            for i, least_enemy_set in enumerate(least_enemy_set_list):
                if enemy_set.issubset(least_enemy_set):
                    least_enemy_set_list[i] = enemy_set
                    is_duplicate = True
                    break
                if least_enemy_set.issubset(enemy_set):
                    is_duplicate = True
                    break
            if not is_duplicate:
                least_enemy_set_list = least_enemy_set_list + [enemy_set]
                
len(least_enemy_set_list)

4474

which is the following (hided)

In [15]:
parsed_least_enemy_set_list = []
for least_enemy_set in least_enemy_set_list:
    workload = sum(get_X(list(set(all_races) - least_enemy_set)))
    efficiency = (len(all_races) - len(list(least_enemy_set)))/workload
    item = " - ".join(list(least_enemy_set)) + ", " + str(workload) + ", " + str(efficiency)
    #print(item)
    parsed_least_enemy_set_list += [item]

3 or less enemies option are listed below

In [16]:
parsed_least_enemy_set_list = []
for least_enemy_set in least_enemy_set_list:
    if len(list(least_enemy_set)) <= 3:
        workload = sum(get_X(list(set(all_races) - least_enemy_set)))
        efficiency = (len(all_races) - len(list(least_enemy_set)))/workload
        item = " - ".join(list(least_enemy_set)) + ", " + str(workload) + ", " + str(efficiency)
        print(item)
        parsed_least_enemy_set_list += [item]

Terran - Yaki - Paranid, 195.7498746292194, 0.06130272125450839


4 enemies

In [17]:
parsed_least_enemy_set_list = []
for least_enemy_set in least_enemy_set_list:
    if len(list(least_enemy_set)) == 4:
        workload = sum(get_X(list(set(all_races) - least_enemy_set)))
        efficiency = (len(all_races) - len(list(least_enemy_set)))/workload
        item = " - ".join(list(least_enemy_set)) + ", " + str(workload) + ", " + str(efficiency)
        print(item)
        parsed_least_enemy_set_list += [item]

Terran - Duke's - Yaki - TerraCorp, 1169.8407341546122, 0.009402989380387044
Teladi - Terran - Yaki - Pirates, 162.41013284002565, 0.06772976419417762
NMMC - Terran - OTAS - Yaki, 824.8106996204908, 0.013336393435561983
Terran - Duke's - Yaki - Arteus, 134248.8546846407, 8.193738431392741e-05
Terran - Yaki - Paranid - Arteus, 44.23056938279498, 0.2486967758610594
Terran - Yaki - Pirates - Arteus, 65.18253733524546, 0.16875685497520648
Terran - OTAS - Duke's - Yaki, 68.01331847910075, 0.1617330288534605
Terran - OTAS - Paranid - Yaki, 44.519618070896385, 0.24708208373402427
Terran - OTAS - Pirates - Yaki, 53.83466312043438, 0.20432931799706308
Terran - Duke's - Paranid - Yaki, 122.18697476218603, 0.0900259624351076
Terran - Duke's - Pirates - Yaki, 39.065113984479844, 0.28158115715137
Terran - Yaki - Paranid - Pirates, 37.78548585409912, 0.29111707184272384
Split - Terran - Paranid - Pirates, 258.260005868024, 0.04259273503471234
Split - Terran - Yaki - Paranid, 47.94917743051851, 0.229

5 enemies but without any major races

In [18]:
parsed_least_enemy_set_list = []
for least_enemy_set in least_enemy_set_list:
    if len(list(least_enemy_set)) == 5 and set(major_races).intersection(least_enemy_set) == set():
        workload = sum(get_X(list(set(all_races) - least_enemy_set)))
        efficiency = (len(all_races) - len(list(least_enemy_set)))/workload
        item = " - ".join(list(least_enemy_set)) + ", " + str(workload) + ", " + str(efficiency)
        print(item)
        parsed_least_enemy_set_list += [item]

TerraCorp - Pirates - Arteus - Duke's - Yaki, 280.24981341472954, 0.03568245016171138
Pirates - Arteus - OTAS - Duke's - Yaki, 59.13128817880325, 0.1691152063144921


## Conclusion

- A simple tool is developed
- All possible combos are given with total workload and efficiency
- Duke's - Pirates - OTAS - Arteus - Strong Arms - Yaki is one of the least enemy approaches to keep good relationship with all 6 major races, and is the most efficient way among